In [1]:
import numpy as np
import os

# Load the numpy array of protein and molecule affinities
protein_affinity_array = np.load('data/kiba/affinity_data.npy')
# Documents/base paper implementation/kiba_dataset/affinity_data.npy

# Path to the aligned protein files
aligned_protein_path = "/home/saeed/Documents/base paper implementation/data/kiba/aln"

# Get list of aligned protein IDs from filenames
aligned_protein_ids = [file.split('.aln')[0] for file in os.listdir(aligned_protein_path) if file.endswith('.aln')]

# Get list of protein IDs from the numpy array
array_protein_ids = protein_affinity_array[:, 0]

# Find proteins in the array but not in the aligned data
proteins_not_in_aligned = np.setdiff1d(array_protein_ids, aligned_protein_ids)

# Find proteins in the aligned data but not in the array
proteins_not_in_array = np.setdiff1d(aligned_protein_ids, array_protein_ids)

# Results
print("Proteins not in aligned data:", proteins_not_in_aligned)
print("Proteins not in array:", proteins_not_in_array)


Proteins not in aligned data: []
Proteins not in array: []


In [ ]:
import os
import numpy as np
import pconsc4
import random
import tensorflow as tf
from multiprocessing import Process
import time

def predict_single_protein(input_file, output_file):
    # Load the pconsc4 model within the separate process
    model = pconsc4.get_pconsc4()

    try:
        print(f'Processing {input_file}')
        pred = pconsc4.predict(model, input_file)  # Predict the contact map
        np.save(output_file, pred['cmap'])  # Save the contact map in .npy format
        print(f'{output_file} saved.')

    except Exception as e:
        print(f'Error processing {input_file}: {e}')
    finally:
        # Clear the Keras session to release memory
        tf.keras.backend.clear_session()
        tf.compat.v1.reset_default_graph()
        # Explicitly delete variables to free memory
        del pred
        del model
        # Add a small delay to ensure resources are released
        time.sleep(6)

def pconsc4Prediction(aligned_protein_path, contact_map_dir):
    # Ensure the contact maps directory exists
    if not os.path.exists(contact_map_dir):
        os.makedirs(contact_map_dir)

    # Get list of aligned protein files
    file_list = os.listdir(aligned_protein_path)
    random.shuffle(file_list)  # Randomize the order of processing

    # Iterate through each alignment file and spawn a separate process
    for file in file_list:
        input_file = os.path.join(aligned_protein_path, file)
        output_file = os.path.join(contact_map_dir, file.split('.aln')[0] + '.npy')  # Store as .npy file

        if os.path.exists(output_file):
            continue  # Skip if the contact map already exists

        # Spawn a new process for each prediction
        p = Process(target=predict_single_protein, args=(input_file, output_file))
        p.start()
        p.join()  # Wait for the process to finish
        p.terminate()  # Explicitly terminate the process to release memory
        # Add a small delay between processes to help with resource management
        time.sleep(1)

        # Monitor memory usage, log it if necessary
        print(f'Completed {file}, checking memory status...')

# TensorFlow GPU memory growth (if applicable)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Paths
aligned_protein_path = "data/kiba/aln"
contact_map_dir = "KibaContactMaps"

# Run the prediction
pconsc4Prediction(aligned_protein_path, contact_map_dir)


Using TensorFlow backend.
/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home

2024-09-30 13:48:11.939411: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 13:48:12.246081: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2024-09-30 13:48:12.248022: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5ac6a0a2ade0 executing computations on platform Host. Devices:
2024-09-30 13:48:12.248110: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #156: KMP_AFFINITY: 8 available OS procs
OMP: Info #1


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2024-09-30 13:48:12.633883: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.


/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


...

OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 1 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L3 cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L2 cache" is equivalent to "core".
OMP: Info #287: KMP_AFFINITY: topology layer "L1 cache" is equivalent to "core".
OMP: Info #192: KMP_AFFINITY: 1 socket x 1 core/socket x 1 thread/core (1 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #254: KMP_AFFINITY: pid 79438 tid 79438 thread 0 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 79438 tid 79471 thread 1 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 79438 tid 79470 thread 2 bound to OS pr

KibaContactMaps/P42338.npy saved.
Completed P42338.aln, checking memory status...






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2024-09-30 14:43:25.779339: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 14:43:25.786430: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2024-09-30 14:43:25.787131: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5ac6a0a2ade0 executing computations on platform Host. Devices:
2024-09-30 14:43:25.787163: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #156: KMP_AFFINITY: 8 available OS procs
OMP: Info #1

/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Processing data/kiba/aln/P50750.aln


In [ ]:
# import os
# import numpy as np
# import pconsc4
# import random
# import tensorflow as tf
# import time
# import gc
# import psutil

# def get_memory_usage():
#     process = psutil.Process(os.getpid())
#     mem_info = process.memory_info()
#     return mem_info.rss / (1024 * 1024)  # Memory usage in MB

# def predict_single_protein(model, input_file, output_file):
#     try:
#         print(f'Processing {input_file}')
#         print(f"Memory usage before prediction: {get_memory_usage():.2f} MB")

#         pred = pconsc4.predict(model, input_file)  # Predict the contact map
#         np.save(output_file, pred['cmap'])  # Save the contact map in .npy format

#         print(f'{output_file} saved.')
#         print(f"Memory usage after prediction: {get_memory_usage():.2f} MB")

#     except Exception as e:
#         print(f'Error processing {input_file}: {e}')
#     finally:
#         # Clear variables to release memory
#         del pred
#         tf.keras.backend.clear_session()
#         tf.compat.v1.reset_default_graph()
#         gc.collect()
#         time.sleep(5)

# def pconsc4Prediction(aligned_protein_path, contact_map_dir):
#     # Ensure the contact maps directory exists
#     if not os.path.exists(contact_map_dir):
#         os.makedirs(contact_map_dir)

#     # Load the pconsc4 model once
#     print("Loading pconsc4 model...")
#     model = pconsc4.get_pconsc4()
#     print("Model loaded.")

#     # Get list of aligned protein files
#     file_list = os.listdir(aligned_protein_path)
#     random.shuffle(file_list)  # Randomize the order of processing

#     # Iterate through each alignment file
#     for idx, file in enumerate(file_list):
#         input_file = os.path.join(aligned_protein_path, file)
#         output_file = os.path.join(contact_map_dir, file.split('.aln')[0] + '.npy')  # Store as .npy file

#         if os.path.exists(output_file):
#             continue  # Skip if the contact map already exists

#         print(f"\nProcessing file {idx + 1}/{len(file_list)}: {file}")
#         predict_single_protein(model, input_file, output_file)

#     # Delete the model after processing all files
#     del model
#     tf.keras.backend.clear_session()
#     tf.compat.v1.reset_default_graph()
#     gc.collect()
#     print("All predictions completed.")

# # TensorFlow GPU memory growth (if applicable)
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("Enabled memory growth for GPU.")
#     except Exception as e:
#         print(f"Could not set memory growth for GPU: {e}")

# # Paths
# aligned_protein_path = "data/kiba/aln"
# contact_map_dir = "KibaContactMaps"

# # Run the prediction
# pconsc4Prediction(aligned_protein_path, contact_map_dir)


In [2]:
# import os
# import numpy as np
# import pconsc4
# import random
# import tensorflow as tf
# from keras import backend as K
# from concurrent.futures import ProcessPoolExecutor

# # Function to predict and save contact map for a single protein
# def predict_single_protein(input_file, output_file):
#     # Load the pconsc4 model inside the separate process
#     model = pconsc4.get_pconsc4()

#     try:
#         print(f'Processing {input_file}')
#         pred = pconsc4.predict(model, input_file)  # Predict the contact map
#         np.save(output_file, pred['cmap'])  # Save the contact map in .npy format
#         print(f'{output_file} saved.')

#     except Exception as e:
#         print(f'Error processing {input_file}: {e}')

#     finally:
#         # Clear the Keras session and reset the TensorFlow graph to release memory
#         K.clear_session()
#         tf.compat.v1.reset_default_graph()

# def pconsc4Prediction(aligned_protein_path, contact_map_dir, num_workers=2):
#     # Ensure the contact maps directory exists
#     if not os.path.exists(contact_map_dir):
#         os.makedirs(contact_map_dir)

#     # Get list of aligned protein files
#     file_list = os.listdir(aligned_protein_path)
#     random.shuffle(file_list)  # Randomize the order of processing

#     tasks = []
#     # Iterate through each alignment file and generate contact maps
#     with ProcessPoolExecutor(max_workers=num_workers) as executor:
#         for file in file_list:
#             input_file = os.path.join(aligned_protein_path, file)
#             output_file = os.path.join(contact_map_dir, file.split('.aln')[0] + '.npy')  # Store as .npy file
            
#             if os.path.exists(output_file):
#                 continue  # Skip if the contact map already exists
            
#             # Submit tasks to process the files in parallel
#             tasks.append(executor.submit(predict_single_protein, input_file, output_file))
        
#         # Wait for all tasks to complete
#         for task in tasks:
#             task.result()  # This ensures we capture any exceptions raised during the execution

# # Paths
# aligned_protein_path = "data/kiba/aln"
# contact_map_dir = "KibaContactMaps"

# # Run the prediction using 4 CPU cores
# pconsc4Prediction(aligned_protein_path, contact_map_dir, num_workers=2)


In [ ]:
#GOOD working ON 1 CPU core !
import os
import numpy as np
import pconsc4
import random
import tensorflow as tf
from multiprocessing import Process

def predict_single_protein(input_file, output_file):
    # Load the pconsc4 model within the separate process
    model = pconsc4.get_pconsc4()

    try:
        print(f'Processing {input_file}')
        pred = pconsc4.predict(model, input_file)  # Predict the contact map
        np.save(output_file, pred['cmap'])  # Save the contact map in .npy format
        print(f'{output_file} saved.')

    except Exception as e:
        print(f'Error processing {input_file}: {e}')
    finally:
        # Clear the Keras session to release memory
        tf.keras.backend.clear_session()
        tf.compat.v1.reset_default_graph()

def pconsc4Prediction(aligned_protein_path, contact_map_dir):
    # Ensure the contact maps directory exists
    if not os.path.exists(contact_map_dir):
        os.makedirs(contact_map_dir)

    # Get list of aligned protein files
    file_list = os.listdir(aligned_protein_path)
    random.shuffle(file_list)  # Randomize the order of processing

    # Iterate through each alignment file and spawn a separate process
    for file in file_list:
        input_file = os.path.join(aligned_protein_path, file)
        output_file = os.path.join(contact_map_dir, file.split('.aln')[0] + '.npy')  # Store as .npy file

        if os.path.exists(output_file):
            continue  # Skip if the contact map already exists

        # Spawn a new process for each prediction
        p = Process(target=predict_single_protein, args=(input_file, output_file))
        p.start()
        p.join()  # Wait for the process to finish before moving to the next file

# Paths
aligned_protein_path = "data/kiba/aln"
contact_map_dir = "KibaContactMaps"

# Run the prediction
pconsc4Prediction(aligned_protein_path, contact_map_dir)


Using TensorFlow backend.
/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home

2024-09-29 09:52:09.085634: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 09:52:09.413154: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2024-09-29 09:52:09.414342: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55710b532440 executing computations on platform Host. Devices:
2024-09-29 09:52:09.414396: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #156: KMP_AFFINITY: 8 available OS procs
OMP: Info #1


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2024-09-29 09:52:09.786311: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.


/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


...

KibaContactMaps/P23443.npy saved.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2024-09-29 10:08:21.331001: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 10:08:21.336546: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2024-09-29 10:08:21.337111: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55710b532440 executing computations on platform Host. Devices:
2024-09-29 10:08:21.337171: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #156: KMP_AFFINITY: 8 available OS procs
OMP: Info #1

/home/saeed/miniconda3/envs/pconsc4_env/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Processing data/kiba/aln/O95819.aln


OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 1 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L3 cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L2 cache" is equivalent to "core".
OMP: Info #287: KMP_AFFINITY: topology layer "L1 cache" is equivalent to "core".
OMP: Info #192: KMP_AFFINITY: 1 socket x 1 core/socket x 1 thread/core (1 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #254: KMP_AFFINITY: pid 12353 tid 12353 thread 0 bound to OS proc set 0


In [ ]:
# import os
# import numpy as np
# import pconsc4
# import random

# def pconsc4Prediction(aligned_protein_path, contact_map_dir):
#     model = pconsc4.get_pconsc4()  # Load the pconsc4 model
    
#     # Ensure the contact maps directory exists
#     if not os.path.exists(contact_map_dir):
#         os.makedirs(contact_map_dir)
    
#     # Get list of aligned protein files
#     file_list = os.listdir(aligned_protein_path)
#     random.shuffle(file_list)  # Randomize the order of processing
    
#     # Iterate through each alignment file and generate contact maps
#     for file in file_list:
#         input_file = os.path.join(aligned_protein_path, file)
#         output_file = os.path.join(contact_map_dir, file.split('.aln')[0] + '.npy')  # Store as .npy file
        
#         if os.path.exists(output_file):
#             continue  # Skip if the contact map already exists
        
#         try:
#             print(f'Processing {input_file}')
#             pred = pconsc4.predict(model, input_file)  # Predict the contact map
#             np.save(output_file, pred['cmap'])  # Save the contact map in .npy format
#             print(f'{output_file} saved.')
#         except Exception as e:
#             print(f'Error processing {file}: {e}')

# # Paths
# aligned_protein_path = "data/kiba/aln"
# contact_map_dir = "KibaContactMaps"

# # Run the prediction
# pconsc4Prediction(aligned_protein_path, contact_map_dir)


In [ ]:
# import os
# import numpy as np
# import pconsc4
# import random
# import tensorflow as tf
# from keras import backend as K

# def pconsc4Prediction(aligned_protein_path, contact_map_dir):
#     model = pconsc4.get_pconsc4()  # Load the pconsc4 model

#     # Ensure the contact maps directory exists
#     if not os.path.exists(contact_map_dir):
#         os.makedirs(contact_map_dir)
    
#     # Get list of aligned protein files
#     file_list = os.listdir(aligned_protein_path)
#     random.shuffle(file_list)  # Randomize the order of processing

#     # Iterate through each alignment file and generate contact maps
#     for file in file_list:
#         input_file = os.path.join(aligned_protein_path, file)
#         output_file = os.path.join(contact_map_dir, file.split('.aln')[0] + '.npy')  # Store as .npy file
        
#         if os.path.exists(output_file):
#             continue  # Skip if the contact map already exists
        
#         try:
#             print(f'Processing {input_file}')
#             pred = pconsc4.predict(model, input_file)  # Predict the contact map
#             np.save(output_file, pred['cmap'])  # Save the contact map in .npy format
#             print(f'{output_file} saved.')
            
#             # Clear Keras session to release memory
#             K.clear_session()
#             tf.compat.v1.reset_default_graph()
#         except Exception as e:
#             print(f'Error processing {file}: {e}')

# # Paths
# aligned_protein_path = "data/kiba/aln"
# contact_map_dir = "KibaContactMaps"

# # Run the prediction
# pconsc4Prediction(aligned_protein_path, contact_map_dir)


In [1]:
# import os
# import numpy as np
# import pconsc4
# import random
# import tensorflow as tf
# from concurrent.futures import ProcessPoolExecutor

# def predict_contact_map(model, input_file, output_file):
#     try:
#         print(f'Processing {input_file}')
        
#         # Predict the contact map
#         pred = pconsc4.predict(model, input_file)
        
#         # Save the contact map in .npy format
#         np.save(output_file, pred['cmap'])
#         print(f'{output_file} saved.')

#     except Exception as e:
#         print(f'Error processing {input_file}: {e}')
#     finally:
#         # Clear TensorFlow session to free up memory after each task
#         tf.keras.backend.clear_session()

# def process_in_parallel(aligned_protein_path, contact_map_dir, num_workers=4):
#     model = pconsc4.get_pconsc4()  # Load the pconsc4 model

#     # Ensure the contact maps directory exists
#     if not os.path.exists(contact_map_dir):
#         os.makedirs(contact_map_dir)

#     # Get list of aligned protein files
#     file_list = os.listdir(aligned_protein_path)
#     random.shuffle(file_list)  # Randomize the order of processing

#     # Use a process pool to handle parallel execution
#     with ProcessPoolExecutor(max_workers=num_workers) as executor:
#         futures = []
#         for file in file_list:
#             input_file = os.path.join(aligned_protein_path, file)
#             output_file = os.path.join(contact_map_dir, file.split('.aln')[0] + '.npy')
            
#             # Skip already processed files
#             if os.path.exists(output_file):
#                 continue

#             # Submit the job to the pool
#             futures.append(executor.submit(predict_contact_map, model, input_file, output_file))

#         # Wait for all futures to complete
#         for future in futures:
#             try:
#                 future.result()  # Ensure any exceptions are caught
#             except Exception as e:
#                 print(f'Error in parallel execution: {e}')

# # Paths
# aligned_protein_path = "/home/saeed/Documents/base paper implementation/data/kiba/aln"
# contact_map_dir = "/home/saeed/Documents/base paper implementation/KibaContactMaps"

# # Run the prediction with parallelism
# process_in_parallel(aligned_protein_path, contact_map_dir, num_workers=4)
